In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np
import re
import seaborn as sns

In [2]:
home_path = '/net/mraid20/export/genie/LabData/Analyses/tomerse/diet_mb/'

In [3]:
mb_names = pd.read_pickle("/net/mraid20/export/genie/LabData/Analyses/tomerse/diet_mb/data/mb_names.pkl")

### SHAP Analyses

In [4]:
phenotypes_mb = pd.read_pickle('/net/mraid20/export/genie/LabData/Analyses/tomerse/diet_mb/data/phenotypes_mb.pkl')

with open('/net/mraid20/export/genie/LabData/Analyses/tomerse/diet_mb/data/target_phenotypes.pkl', 'rb') as f:
    target_phenotypes = pickle.load(f)

target_phenotypes = sum(target_phenotypes, [])

with open('/net/mraid20/export/genie/LabData/Analyses/tomerse/diet_mb/data/microbial_features.pkl', 'rb') as f:
    microbial_features = pickle.load(f)

In [5]:
len(target_phenotypes)

30

In [6]:
len(microbial_features)

726

In [7]:
def mean_abs_shap(shap_orig_path, output_path, targets):
    mean_abs_shap = pd.Series(np.zeros(len(microbial_features)))
    print("Reading shap values...")
    shap_orig = pd.read_pickle(shap_orig_path)
    print("shap_orig shape:", shap_orig.shape)
    for i in range(0, len(targets)):
    # # each row is a sample, each column is a feature, value is shap value
        print(i)
        shap = shap_orig.iloc[i]

        shap = shap[shap.notnull() & shap.apply(lambda x: x is not None)]

        shap_df = pd.DataFrame(shap.tolist())
        to_concat = shap_df.abs().mean()
        to_concat = pd.DataFrame(to_concat)
        to_concat.columns = [str(i)]  # Specify your desired column name here
        mean_abs_shap = pd.concat([mean_abs_shap, to_concat], axis=1)
    mean_abs_shap = mean_abs_shap.iloc[:, 1:]
    mean_abs_shap.to_pickle(output_path)
    return mean_abs_shap

In [8]:
# mean_abs_shap_mb_phenotypes = mean_abs_shap(
#     shap_orig_path = home_path + f"/data/output_SHAP_XGB_mb_phenotypes.pkl",
#     output_path = home_path + f"data/mean_abs_shap_XGB_mb_phenotypes.pkl",
#     targets = target_phenotypes
# )

#### What are the highest mean abs shap values?

In [9]:
mean_abs_shap_mb_phenotypes = pd.read_pickle(home_path + f"data/mean_abs_shap_XGB_mb_phenotypes.pkl")
mean_abs_shap_mb_phenotypes

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,3.849538e-03,0.034195,0.269884,0.013744,1.085895e-02,2.467648,0.082778,0.000467,3.480312e-03,3.268748,...,1.065717,0.131470,0.031324,0.039666,4.431233,2.246560,3.490731,0.017269,4.162044,0.414424
1,1.147486e-02,0.144190,0.964736,0.044676,3.905960e-03,19.634579,3.996505,0.805116,2.823399e-02,4.920090,...,0.644789,0.252202,0.751818,0.668299,2.666693,2.683406,1.095131,0.172274,0.388985,4.279988
2,0.000000e+00,0.000002,0.000000,0.000000,7.770358e-06,0.000000,0.000064,0.000000,0.000000e+00,0.000302,...,0.000000,0.000077,0.000000,0.000000,0.000000,0.073394,0.022488,0.003098,0.000000,0.005694
3,0.000000e+00,0.000000,0.000045,0.000003,0.000000e+00,0.000350,0.000000,0.000064,6.739236e-07,0.000539,...,0.008364,0.000127,0.000030,0.000529,0.049551,0.006954,0.009794,0.000350,0.000000,0.000000
4,1.348315e-06,0.000023,0.001001,0.000003,2.323638e-05,0.000000,0.000000,0.000000,2.836844e-06,0.002286,...,0.015985,0.001336,0.002458,0.000291,0.016843,0.062951,0.000000,0.007764,0.035827,0.041721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,0.000000e+00,0.000000,0.000050,0.000000,0.000000e+00,0.000000,0.000100,0.000103,0.000000e+00,0.001502,...,0.018208,0.000000,0.001508,0.000000,0.000000,0.000000,0.013530,0.000000,0.048451,0.000000
722,3.078895e-07,0.000000,0.000179,0.000000,5.894716e-07,0.000364,0.000000,0.000000,9.801472e-07,0.000000,...,0.012889,0.000959,0.000162,0.000685,0.000000,0.223402,0.000000,0.004218,0.000000,0.000000
723,0.000000e+00,0.000000,0.000394,0.000000,5.682145e-06,0.001428,0.000381,0.000117,0.000000e+00,0.001542,...,0.001054,0.000145,0.001589,0.000000,0.000000,0.008312,0.000000,0.000882,0.026846,0.000000
724,2.005504e-06,0.000375,0.005382,0.000025,4.756664e-05,0.000930,0.000550,0.000375,5.450120e-05,0.000785,...,0.055058,0.001933,0.000792,0.022195,0.054955,0.419346,0.071767,0.022216,0.614460,0.142124


In [10]:
# Some predictions are NA if the training set has prevalence >95%
mean_abs_shap_mb_phenotypes.dropna(axis=1)
# mean_shap_lgbm_abundance.dropna(axis=1)

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,3.849538e-03,0.034195,0.269884,0.013744,1.085895e-02,2.467648,0.082778,0.000467,3.480312e-03,3.268748,...,1.065717,0.131470,0.031324,0.039666,4.431233,2.246560,3.490731,0.017269,4.162044,0.414424
1,1.147486e-02,0.144190,0.964736,0.044676,3.905960e-03,19.634579,3.996505,0.805116,2.823399e-02,4.920090,...,0.644789,0.252202,0.751818,0.668299,2.666693,2.683406,1.095131,0.172274,0.388985,4.279988
2,0.000000e+00,0.000002,0.000000,0.000000,7.770358e-06,0.000000,0.000064,0.000000,0.000000e+00,0.000302,...,0.000000,0.000077,0.000000,0.000000,0.000000,0.073394,0.022488,0.003098,0.000000,0.005694
3,0.000000e+00,0.000000,0.000045,0.000003,0.000000e+00,0.000350,0.000000,0.000064,6.739236e-07,0.000539,...,0.008364,0.000127,0.000030,0.000529,0.049551,0.006954,0.009794,0.000350,0.000000,0.000000
4,1.348315e-06,0.000023,0.001001,0.000003,2.323638e-05,0.000000,0.000000,0.000000,2.836844e-06,0.002286,...,0.015985,0.001336,0.002458,0.000291,0.016843,0.062951,0.000000,0.007764,0.035827,0.041721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
721,0.000000e+00,0.000000,0.000050,0.000000,0.000000e+00,0.000000,0.000100,0.000103,0.000000e+00,0.001502,...,0.018208,0.000000,0.001508,0.000000,0.000000,0.000000,0.013530,0.000000,0.048451,0.000000
722,3.078895e-07,0.000000,0.000179,0.000000,5.894716e-07,0.000364,0.000000,0.000000,9.801472e-07,0.000000,...,0.012889,0.000959,0.000162,0.000685,0.000000,0.223402,0.000000,0.004218,0.000000,0.000000
723,0.000000e+00,0.000000,0.000394,0.000000,5.682145e-06,0.001428,0.000381,0.000117,0.000000e+00,0.001542,...,0.001054,0.000145,0.001589,0.000000,0.000000,0.008312,0.000000,0.000882,0.026846,0.000000
724,2.005504e-06,0.000375,0.005382,0.000025,4.756664e-05,0.000930,0.000550,0.000375,5.450120e-05,0.000785,...,0.055058,0.001933,0.000792,0.022195,0.054955,0.419346,0.071767,0.022216,0.614460,0.142124


In [11]:
# Compute full correlation matrix
full_corr = phenotypes_mb[microbial_features  + target_phenotypes].corr(method='spearman')

# Extract only the correlations between features and significant_targets
correlation_matrix = full_corr.loc[microbial_features , target_phenotypes]
correlation_matrix

,vat_fat_ratio,android/gynoid ratio,fat_mass_index,waist_hips_ratio,waist_height_ratio,mean_hand_grip,ALM,ALMI,ALM_BMI,sitting_systolic_bp_mean,...,bt__glucose,bt__mcv,bt__hemoglobin,bt__uric_acid,bt__total_cholesterol,bt__triglycerides,bt__non_hdl_cholesterol,triglyceride_to_hdl_ratio,bt__ldl_cholesterol,bt__hdl_cholesterol
age,0.197467,0.121299,0.145308,0.155019,0.206406,-0.165548,-0.082209,-0.044000,-0.150145,0.241097,...,0.158998,0.095971,-0.007819,-0.000540,0.160418,0.094637,0.116085,0.021442,0.179395,0.133199
sex,0.611425,0.617533,-0.363223,0.586542,0.136065,0.816135,0.821549,0.724635,0.763677,0.360860,...,0.116641,-0.114670,0.687192,0.604530,-0.117040,0.125318,0.038217,0.241474,-0.024781,-0.431714
Duodenibacillus intestinigallinarum,0.000255,0.013277,0.010185,0.010711,0.000069,-0.014882,0.001315,-0.008983,-0.007930,0.015895,...,0.019652,0.014527,-0.005077,-0.001447,0.018920,-0.008244,0.019487,-0.014632,0.020139,0.000413
Duodenibacillus intestinavium,0.022842,0.012118,0.015282,0.004344,0.006904,0.003378,0.001048,-0.000102,-0.006270,0.021782,...,0.037494,0.000371,0.004942,0.020954,0.010486,0.014743,0.031812,0.014583,0.023079,-0.029473
Duodenibacillus massiliensis,0.016750,0.021291,-0.039526,0.014888,-0.004005,0.047277,0.054914,0.052121,0.061561,-0.004609,...,-0.015688,0.023008,0.011535,0.016839,-0.023370,-0.012824,-0.014229,-0.003267,-0.012194,-0.036429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Duodenibacillus sp900544335,0.015642,0.011505,-0.026276,0.033532,0.019012,0.035698,0.049577,0.053286,0.044709,0.001972,...,0.027655,0.018437,0.050293,0.008630,0.009327,-0.000311,0.000224,-0.002266,-0.012656,0.005284
Aphodousia faecalis,0.022432,0.018250,-0.001644,0.022443,0.010175,0.019826,0.019409,0.015438,0.017483,0.025806,...,0.034780,0.016656,0.014844,-0.023249,0.006717,-0.029307,0.007932,-0.030629,0.002078,0.009053
Parasutterella gallistercoris,0.007770,-0.001317,0.046588,-0.012425,0.020505,-0.035228,-0.031429,-0.025124,-0.050733,-0.002351,...,0.012844,-0.030028,-0.019975,0.006291,0.021086,0.019205,0.017210,0.006490,0.007172,0.022697
Parasutterella excrementihominis,-0.018408,-0.020265,0.037443,-0.027380,-0.012671,-0.036770,-0.046108,-0.048062,-0.048824,-0.026561,...,0.038284,0.003345,-0.005951,-0.002514,0.005262,0.010837,0.002390,0.016946,0.023024,-0.007402


In [12]:
signed_mean_abs_shap = np.sign(correlation_matrix)
signed_mean_abs_shap.index = mean_abs_shap_mb_phenotypes.index
signed_mean_abs_shap.columns = mean_abs_shap_mb_phenotypes.columns
signed_mean_abs_shap = mean_abs_shap_mb_phenotypes * signed_mean_abs_shap

signed_mean_abs_shap.index = microbial_features
signed_mean_abs_shap.columns = target_phenotypes

signed_mean_abs_shap.to_pickle('/net/mraid20/export/genie/LabData/Analyses/tomerse/diet_mb/data/diet_intervention/directional_phenotypes_shap.pkl')
signed_mean_abs_shap

,vat_fat_ratio,android/gynoid ratio,fat_mass_index,waist_hips_ratio,waist_height_ratio,mean_hand_grip,ALM,ALMI,ALM_BMI,sitting_systolic_bp_mean,...,bt__glucose,bt__mcv,bt__hemoglobin,bt__uric_acid,bt__total_cholesterol,bt__triglycerides,bt__non_hdl_cholesterol,triglyceride_to_hdl_ratio,bt__ldl_cholesterol,bt__hdl_cholesterol
age,3.849538e-03,0.034195,0.269884,0.013744,1.085895e-02,-2.467648,-0.082778,-0.000467,-3.480312e-03,3.268748,...,1.065717,0.131470,-0.031324,-0.039666,4.431233,2.246560,3.490731,0.017269,4.162044,0.414424
sex,1.147486e-02,0.144190,-0.964736,0.044676,3.905960e-03,19.634579,3.996505,0.805116,2.823399e-02,4.920090,...,0.644789,-0.252202,0.751818,0.668299,-2.666693,2.683406,1.095131,0.172274,-0.388985,-4.279988
Duodenibacillus intestinigallinarum,0.000000e+00,0.000002,0.000000,0.000000,7.770358e-06,-0.000000,0.000064,-0.000000,-0.000000e+00,0.000302,...,0.000000,0.000077,-0.000000,-0.000000,0.000000,-0.073394,0.022488,-0.003098,0.000000,0.005694
Duodenibacillus intestinavium,0.000000e+00,0.000000,0.000045,0.000003,0.000000e+00,0.000350,0.000000,-0.000064,-6.739236e-07,0.000539,...,0.008364,0.000127,0.000030,0.000529,0.049551,0.006954,0.009794,0.000350,0.000000,-0.000000
Duodenibacillus massiliensis,1.348315e-06,0.000023,-0.001001,0.000003,-2.323638e-05,0.000000,0.000000,0.000000,2.836844e-06,-0.002286,...,-0.015985,0.001336,0.002458,0.000291,-0.016843,-0.062951,-0.000000,-0.007764,-0.035827,-0.041721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Duodenibacillus sp900544335,0.000000e+00,0.000000,-0.000050,0.000000,0.000000e+00,0.000000,0.000100,0.000103,0.000000e+00,0.001502,...,0.018208,0.000000,0.001508,0.000000,0.000000,-0.000000,0.013530,-0.000000,-0.048451,0.000000
Aphodousia faecalis,3.078895e-07,0.000000,-0.000179,0.000000,5.894716e-07,0.000364,0.000000,0.000000,9.801472e-07,0.000000,...,0.012889,0.000959,0.000162,-0.000685,0.000000,-0.223402,0.000000,-0.004218,0.000000,0.000000
Parasutterella gallistercoris,0.000000e+00,-0.000000,0.000394,-0.000000,5.682145e-06,-0.001428,-0.000381,-0.000117,-0.000000e+00,-0.001542,...,0.001054,-0.000145,-0.001589,0.000000,0.000000,0.008312,0.000000,0.000882,0.026846,0.000000
Parasutterella excrementihominis,-2.005504e-06,-0.000375,0.005382,-0.000025,-4.756664e-05,-0.000930,-0.000550,-0.000375,-5.450120e-05,-0.000785,...,0.055058,0.001933,-0.000792,-0.022195,0.054955,0.419346,0.071767,0.022216,0.614460,-0.142124


In [13]:
# def process_mean_signed_shap(mean_abs_shap_df, correlation_matrix, all_features, top_n=20):
#     # Compute mean absolute SHAP values and signed mean SHAP values across targets
#     mean_abs_shap_df['mean_abs_shap'] = mean_abs_shap_df.mean(axis=1)
#     correlation_matrix['mean_corr'] = correlation_matrix.mean(axis=1)

#     # Compute directional absolute SHAP
#     direction = np.sign(correlation_matrix['mean_corr']).reset_index(drop=True)
#     directional_abs_shap = mean_abs_shap_df['mean_abs_shap'] * direction

#     # Full feature names
#     feature_names = pd.Series(all_features)

#     # Create full shap data for return
#     shap_data_all = pd.DataFrame({
#         'Feature': feature_names,
#         'Directional Mean Absolute SHAP Value': directional_abs_shap.values
#     })

#     # Select top and bottom for plotting only
#     top_features = directional_abs_shap.sort_values(ascending=False).head(top_n)
#     bottom_features = directional_abs_shap.sort_values(ascending=True).head(top_n)
#     combined = pd.concat([top_features, bottom_features])

#     # Plotting data
#     shap_data_plot = shap_data_all.loc[combined.index]
#     shap_data_plot = shap_data_plot.sort_values(by='Directional Mean Absolute SHAP Value')

#     # Colors
#     colors = ['cadetblue' if val > 0 else 'salmon' for val in shap_data_plot['Directional Mean Absolute SHAP Value']]

#     # Plot
#     plt.figure(figsize=(8, 8), dpi=300)
#     plt.barh(shap_data_plot['Feature'], shap_data_plot['Directional Mean Absolute SHAP Value'], color=colors, edgecolor='none')
#     plt.xlabel('Signed Mean Absolute SHAP Value', fontsize=14)
#     plt.ylabel('Features', fontsize=14)
#     plt.title('Top and Bottom Features by Directional SHAP Value', fontsize=16)
#     plt.tight_layout()
#     plt.show()

#     return shap_data_all

# shap_data_all = process_mean_signed_shap(mean_abs_shap_mb_phenotypes, correlation_matrix, microbial_features, top_n=20)